In [22]:
import os
import warnings

from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA

from dotenv import load_dotenv

# Ignore all warnings (not recommended in production code)
warnings.filterwarnings("ignore")

In [2]:
# Get your API keys from environment variables
load_dotenv()
openai_key = os.getenv("OPENAI_API_KEY")
huggingface_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")
verbose = False

### Load VectorDB from disk:

In [3]:
# load from disk
embedding = OpenAIEmbeddings()
persist_directory = 'db'
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

### Query Documents similar to Query

In [4]:
docs = vectordb.similarity_search("Dear Fellow Scholars, this is Two Minute Papers with Dr", k = 3)
docs

[Document(page_content="Dear Fellow Scholars, this is Two Minute Papers with Dr. KÃ¡roly Zsolnai-FehÃ©r. Today, we are going to have a look at an image creation AI called Perfusion that is completely out of this world. Imagine a text-to-image AI like Stable Diffusion or Meat Journey as a chef that can cook any dish. However, sometimes, we like the dish we get, but we would like to introduce our own ingredients to it. And normally, if we write a new recipe, a new text prompt, we get a completely new image, a new dish. However, that is not what we want. What we want is the previous dish with a new ingredient. Let's have a look at an example. We have photos of our favorite teddy bear, and we would like it to create an image of it playing with a ball in the water. A previous technique doesâ€¦uh-oh. I have already looked at this image, and I have bad news. It's not going to be pretty. Are you ready? Okay, here you go! Whoa! It seems to have fallen apart, and there is no ball anywhere to be 

# Build Simple RAG Chain

In [11]:
# I consider only top k=1 candidate as chunksize is large
# In real app we would reduce chunk size to smaller chunks 
retriever = vectordb.as_retriever(search_kwargs={"k": 1}) 
docs = retriever.get_relevant_documents("Someone said this is X Minute Papers. How many minutes were they?")
docs

[Document(page_content="Dear Fellow Scholars, this is Two Minute Papers with Dr. KÃ¡roly Zsolnai-FehÃ©r. Today, we are going to have a look at an image creation AI called Perfusion that is completely out of this world. Imagine a text-to-image AI like Stable Diffusion or Meat Journey as a chef that can cook any dish. However, sometimes, we like the dish we get, but we would like to introduce our own ingredients to it. And normally, if we write a new recipe, a new text prompt, we get a completely new image, a new dish. However, that is not what we want. What we want is the previous dish with a new ingredient. Let's have a look at an example. We have photos of our favorite teddy bear, and we would like it to create an image of it playing with a ball in the water. A previous technique doesâ€¦uh-oh. I have already looked at this image, and I have bad news. It's not going to be pretty. Are you ready? Okay, here you go! Whoa! It seems to have fallen apart, and there is no ball anywhere to be 

In [18]:
# Simple function to add some meta data for fact checking
def process_llm_response(llm_response):
    print(f"AI Response:\n{llm_response['result']}")
    print('\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(), 
                                  chain_type="stuff", 
                                  retriever=retriever, 
                                  return_source_documents=True)

In [21]:
query = "Someone said this is X Minute Papers. How many minutes were they and what is the name of the Dr?"
llm_response = qa_chain(query)
process_llm_response(llm_response)

AI Response:
 This is Two Minute Papers with Dr. Károly Zsolnai-Fehér.

Sources:
documents\NVIDIAs_New_AI_Text_To_Image_Supercharged.txt
